The below UNET code is inspired by this tutorial: https://www.tensorflow.org/tutorials/images/segmentation

# Importing libraries

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

from tensorflow_examples.models.pix2pix import pix2pix

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from IPython.display import clear_output
import matplotlib.pyplot as plt
import os
import shutil
import numpy as np
import random
from random import shuffle
from PIL import Image

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Creating Image Data Generators

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.__version__

'2.2.0'

In [ ]:
sz = (128, 128)
BATCH_SIZE = 20

In [ ]:
# note that for the directories, keras data generators expect a url that points to the folder
# that is located at the second level above the images/masks
# folder --> directory fed to keras generator
#### folder --> sub-directory, usually these stands to the number of clases for the classification model, since it's a segmentation task, it's only one folder
####### images or masks

# directories of the sugarbeet dataset
rgb_images_dir = '/home/path/to/images'
annotations_color_dir = '/home/path/to/masks'

In [ ]:
# we create two instances with the same arguments
data_gen_args = dict(
                     horizontal_flip=True,
                     vertical_flip=True,
                     rescale=1./255,
                     validation_split=0.1
                     )

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

#Splitting training and validation without applying same transformations to both datasets:
#https://stackoverflow.com/questions/53037510/can-flow-from-directory-get-train-and-validation-data-from-the-same-directory-in
#https://stackoverflow.com/questions/42443936/keras-split-train-test-set-when-using-imagedatagenerator

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
train_image_generator = image_datagen.flow_from_directory(
    rgb_images_dir,
    target_size = sz,
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=True,
    subset='training',
    seed=seed)
train_mask_generator = mask_datagen.flow_from_directory(
    annotations_color_dir,
    target_size = sz,
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=True,
    subset='training',
    seed=seed)

val_image_generator = image_datagen.flow_from_directory(
    rgb_images_dir,
    target_size = sz,
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=True,
    subset='validation',
    seed=seed)
val_mask_generator = mask_datagen.flow_from_directory(
    annotations_color_dir,
    target_size = sz,
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=True,
    subset='validation',
    seed=seed)

 #about resetting test generators
#https://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720


Found 4224 images belonging to 1 classes.
Found 4224 images belonging to 1 classes.
Found 469 images belonging to 1 classes.
Found 469 images belonging to 1 classes.


In [ ]:
def normalize(images):
  normalized_images = []
  for i in range(len(images)):
    image = images[i]
    max_val = np.ceil(np.max(image))
    image = image/max_val
    normalized_images.append(image)
  return normalized_images

In [ ]:
#If this doesn't work, applying mask changes
#https://github.com/keras-team/keras-preprocessing/issues/125
def prepare_mask(masks):
  new_masks = [] 
  for i in range (np.shape(masks)[0]):
    mask_datapoint = masks[i]
    r = mask_datapoint[:,:,0]
    g = mask_datapoint[:,:,1]
    b = mask_datapoint[:,:,2]
    r[r<=(150/255)]=0 
    r[r>(150/255)]=3
    g[g<=(150/255)]=0 
    g[g>(150/255)]=1
    b[b<=(150/255)]=0 
    b[b>(150/255)]=2
    x_centroid=[]
    y_centroid=[]
    for x in range(b.shape[0]):
      for y in range(b.shape[1]):
        if b[x,y]==2:
          x_centroid.append(x)
          y_centroid.append(y)
    
    for i in range(len(x_centroid)):
      x=x_centroid[i]
      y=y_centroid[i]
      augm=1
      x_augm_plus=x+augm
      x_augm_min=x-augm
      y_augm_plus=y+augm
      y_augm_min=y-augm
      
      if(x_augm_plus<b.shape[0]):
        b[x_augm_plus,y]=2
      if(x_augm_min>0):
        b[x_augm_min,y]=2
      if(y_augm_plus<b.shape[1]):
        b[x,y_augm_plus]=2
      if(y_augm_min>0):
        b[x,y_augm_min]=2
      
      if(x_augm_plus>0 and x_augm_plus<b.shape[0] and y_augm_plus>0 and y_augm_plus<b.shape[1]):
        b[x_augm_plus,y_augm_plus]=2
      if(x_augm_plus>0 and x_augm_plus<b.shape[0] and y_augm_min>0 and y_augm_min<b.shape[1]):
        b[x_augm_plus,y_augm_min]=2
      if(x_augm_min>0 and x_augm_min<b.shape[0] and y_augm_plus>0 and y_augm_plus<b.shape[1]):
        b[x_augm_min,y_augm_plus]=2
      if(x_augm_min>0 and x_augm_min<b.shape[0] and y_augm_min>0 and y_augm_min<b.shape[1]):
        b[x_augm_min,y_augm_min]=2

    merged = np.maximum.reduce([r,g,b])
    new_masks.append(merged)
  new_masks = np.expand_dims(new_masks,3)
  return new_masks

In [ ]:
def my_image_mask_generator(image_generator, mask_generator):
    train_generator = zip(image_generator, mask_generator)
    for (imgs, msks) in train_generator:
        msks_total = prepare_mask(msks)
        yield (imgs, msks_total)

In [ ]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask', 'Fourth image']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [ ]:
 #for joining generators
 #https://stackoverflow.com/questions/3211041/how-to-join-two-generators-in-python
 #https://stackoverflow.com/questions/49404993/keras-how-to-use-fit-generator-with-multiple-inputs/49405175#comment89680557_49405175
my_train_generator = my_image_mask_generator(train_image_generator, train_mask_generator)
my_val_generator = my_image_mask_generator(val_image_generator, val_mask_generator)

In [ ]:
x,y = next(my_train_generator)
sample_image = x[2]
sample_mask = y[2]

In [ ]:
z,w = next(my_val_generator)

In [ ]:
for i in range(len(z)):
  sample_image = z[i]
  sample_mask = w[i]
  display([sample_image,sample_mask])

# Define the model
The model being used here is a modified U-Net. A pretrained model (MobileNetV2) is used as the encoder. The decoder will be the upsample block already implemented in TensorFlow Examples in the [Pix2pix tutorial](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/pix2pix/pix2pix.py). 

In [ ]:
OUTPUT_CHANNELS = 4

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)
base_model.summary()
# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]
# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

In [ ]:
tf.keras.utils.plot_model(down_stack, show_shapes=True)

The decoder/upsampler is simply a series of upsample blocks implemented in TensorFlow examples.

In [ ]:
up_stack_crop = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

In [ ]:
def unet_model(output_channels, inputs):
  #x = inputs

  # Downsampling through the model
  skips = down_stack(inputs)
  print('skips', skips)
  last_downsampled_layer = skips[-1]
  #print('last_downsampled_layer', last_downsampled_layer)
  x = last_downsampled_layer
  skips = list(reversed(skips[:-1]))

  ##################################################
  ############# Crop/Weed branch ###################
  ##################################################

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack_crop, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  # This is the last layer of the model
  last = tf.keras.layers.Conv2DTranspose(
      output_channels, 3, strides=2,
      padding='same')  #64x64 -> 128x128
  x = last(x)

  #classification 
  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
inputs = tf.keras.layers.Input(shape=[128, 128, 3])
model = unet_model(OUTPUT_CHANNELS, inputs)

skips [<tf.Tensor 'model/Identity:0' shape=(None, 64, 64, 96) dtype=float32>, <tf.Tensor 'model/Identity_1:0' shape=(None, 32, 32, 144) dtype=float32>, <tf.Tensor 'model/Identity_2:0' shape=(None, 16, 16, 192) dtype=float32>, <tf.Tensor 'model/Identity_3:0' shape=(None, 8, 8, 576) dtype=float32>, <tf.Tensor 'model/Identity_4:0' shape=(None, 4, 4, 320) dtype=float32>]


In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

# Define the IoU metric

In [ ]:
def mean_iou(y_true, y_pred):
  y_pred = K.cast(create_mask(y_pred), 'float32')
  inter = tf.math.count_nonzero(tf.logical_and(tf.not_equal(y_true, 0), tf.equal(y_true,y_pred)))
  union = tf.math.count_nonzero(tf.add(y_true, y_pred))
  my_iou = tf.cast(inter/union, 'float32')
  return my_iou

# Compile the model
The network is trying to assign each pixel a label. In the true segmentation mask, each pixel has either a {0,1,2}. The network here is outputting three channels. Essentially, each channel is trying to learn to predict a class.

In [ ]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy",mean_iou]
              )

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
model (Model)                   [(None, 64, 64, 96), 1841984     input_2[0][0]                    
__________________________________________________________________________________________________
sequential (Sequential)         (None, 8, 8, 512)    1476608     model[1][4]                      
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 8, 8, 1088)   0           sequential[0][0]                 
                                                                 model[1][3]                

# Viewing the model performance before training the trainable parameters

Using the output of the network, the label assigned to the pixel is the channel with the highest value. This is what the create_mask function is doing.

In [ ]:
from tensorflow.keras import backend as K

In [ ]:
threshold = 0.5

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [ ]:
def show_predictions(dataset=None, num=1):
  if dataset:
    pred_x, pred_y = next(dataset)
    for i in range (num):
      image, mask = pred_x[i], pred_y[i]

      pred_mask = model.predict(image[tf.newaxis, ...])
      my_iou_crop =  mean_iou(mask, pred_mask)
      print('My Mean IoU for sample image: ', tf.keras.backend.eval(my_iou_crop))

      pred_mask = create_mask(pred_mask)
      print(np.unique(pred_mask), pred_mask.shape)
      
      display([image, mask, pred_mask])
  else:
    pred_mask = model.predict(sample_image[tf.newaxis, ...])
    print('after applying mask, uniques of predicted mask:', np.unique(pred_mask), pred_mask.shape)
    print('uniques of sample mask', np.unique(sample_mask))
    my_iou =  mean_iou(sample_mask, pred_mask)
    print('My Mean IoU for sample image: ', tf.keras.backend.eval(my_iou))

    pred_mask = create_mask(pred_mask)
    display([sample_image, sample_mask, pred_mask])

# Define the Callback

In [ ]:
LAST_EPOCH=312
class ShowPredictions(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+LAST_EPOCH+1))

class SaveWeights(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    model.save_weights("/home/path/to/weights/epoch"+str(epoch+LAST_EPOCH+1)+".h5")

In [ ]:
def build_callbacks():
  callbacks = [SaveWeights(),
               ShowPredictions()]
               
  return callbacks

# Load a saved model

In [ ]:
model.load_weights("/home/path/to/weights/epoch"+str(LAST_EPOCH)+".h5")

In [ ]:
show_predictions()

# Train the model

In [ ]:
#change these values according to the number of images
TOTAL_TRAIN_IMAGES=4224
TOTAL_VAL_IMAGES=469

STEPS_PER_EPOCH = np.floor(TOTAL_TRAIN_IMAGES/BATCH_SIZE)
VALIDATION_STEPS = np.floor(TOTAL_VAL_IMAGES/BATCH_SIZE)
print(STEPS_PER_EPOCH, VALIDATION_STEPS)

211.0 23.0


In [ ]:
model_history = model.fit_generator(my_train_generator,epochs=100,
                                    steps_per_epoch=STEPS_PER_EPOCH,
                                    validation_data=my_val_generator,
                                    validation_steps=VALIDATION_STEPS,
                                    callbacks=build_callbacks()
                                    )

In [ ]:
loss = model_history.history['mean_iou']
val_loss = model_history.history['val_mean_iou']

epochs = range(100)

plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 0.2])
plt.legend()
plt.show()

# Make predictions

See how the system behaves on a set of images

In [ ]:
next(my_val_generator)

In [ ]:
show_predictions(my_val_generator,num=10)

#Save final model

In [ ]:
tf.keras.models.save_model(model,"/home//home/path/to/weights/final"+str(LAST_EPOCH)+".pb")

## Verifying the performance of the model

In [ ]:
from sklearn.metrics import confusion_matrix  
import numpy as np

def centroid_accuracy(y_true,y_pred):
 # ytrue, ypred is a flatten vector
  y_true = y_true.flatten()
  y_pred=create_mask(y_pred).numpy().flatten()
  current = confusion_matrix(y_true, y_pred, labels=[0, 1,2,3])
  acc= current[2,2]/np.nansum(current[2])
  return acc

- Verifying on train data

In [ ]:
count=0
avg_train_mean_iou=0
avg_centroid_acc=0
data_size_iou=0
data_size_acc=0
for i in range(int(STEPS_PER_EPOCH)) :
    x,y=next(my_train_generator)
    for i in range (x.shape[0]):
      
      image, mask = x[i], y[i]
      pred_mask = model.predict(image[tf.newaxis, ...])
      my_iou_crop =  mean_iou(mask, pred_mask).numpy()
      centroid_acc=centroid_accuracy(mask, pred_mask)
      if not np.isnan(my_iou_crop):
          data_size_iou+=1
          avg_train_mean_iou+= my_iou_crop
      if not np.isnan(centroid_acc):
          avg_centroid_acc+=centroid_acc
          data_size_acc+=1
      if np.isnan(my_iou_crop):
          count+=1
          print('My Mean IoU for sample image: ', tf.keras.backend.eval(my_iou_crop))

          pred_mask = create_mask(pred_mask)
          print(np.unique(pred_mask), pred_mask.shape)

          display([image, mask, pred_mask])
avg_train_mean_iou/=data_size_iou
avg_centroid_acc/=data_size_acc
print("avg train mean iou:",avg_train_mean_iou)
print("pics with nan iou :",count)
print("centroid accuracy:",avg_centroid_acc)

- Verifying on validation data

In [ ]:
count=0
avg_val_mean_iou=0
for i in range(int(VALIDATION_STEPS)) :
    x,y=next(my_val_generator)
    for i in range (x.shape[0]):
      image, mask = x[i], y[i]
      pred_mask = model.predict(image[tf.newaxis, ...])
      my_iou_crop =  mean_iou(mask, pred_mask)
      avg_val_mean_iou+= my_iou_crop
      if(my_iou_crop<0.1):
          count+=1
          print('My Mean IoU for sample image: ', tf.keras.backend.eval(my_iou_crop))

          pred_mask = create_mask(pred_mask)
          print(np.unique(pred_mask), pred_mask.shape)

          display([image, mask, pred_mask])
avg_val_mean_iou/=TOTAL_VAL_IMAGES
print("avg val mean iou:",avg_val_mean_iou)
print("pics with iou less than 0.1:",count)

In [ ]:
count=0
avg_train_mean_iou=0
avg_centroid_acc=0
data_size_iou=0
data_size_acc=0
count2=0
for i in range(int(STEPS_PER_EPOCH)) :
    x,y=next(my_train_generator)
    for i in range (x.shape[0]):
      
      image, mask = x[i], y[i]
      pred_mask = model.predict(image[tf.newaxis, ...])
      my_iou_crop =  mean_iou(mask, pred_mask).numpy()
      centroid_acc=centroid_accuracy(mask, pred_mask)
      print(centroid_acc)
      if not np.isnan(my_iou_crop):
          data_size_iou+=1
          avg_train_mean_iou+= my_iou_crop
      if not np.isnan(centroid_acc):
          avg_centroid_acc+=centroid_acc
          data_size_acc+=1
      if np.isnan(my_iou_crop):
          count+=1
          print('My Mean IoU for sample image: ', tf.keras.backend.eval(my_iou_crop))

          pred_mask = create_mask(pred_mask)
          print(np.unique(pred_mask), pred_mask.shape)

          display([image, mask, pred_mask])
      if my_iou_crop<0.1:
          count2+=1
          print('My Mean IoU for sample image: ', tf.keras.backend.eval(my_iou_crop))

          pred_mask = create_mask(pred_mask)
          print(np.unique(pred_mask), pred_mask.shape)

          display([image, mask, pred_mask])

avg_train_mean_iou/=data_size_iou
avg_centroid_acc/=data_size_acc
print("avg train mean iou:",avg_train_mean_iou)
print("pics with iou less than 0.1:",count2)
print("pics with nan iou:",count)
print("centroid accuracy:",avg_centroid_acc)

## Adding the 4th channel

In [ ]:
predictions = tf.keras.layers.Conv2DTranspose(
      4, 3, strides=2,
      padding='same')(model.layers[-2].output)
model = tf.keras.Model(inputs=inputs, outputs=predictions)
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy",mean_iou]
              )

model.summary()

In [ ]:
weights_list = model.get_weights()
weights_list[0][1][0][0]==new_model.get_weights()[0][1][0][0]